## goal : extract the density population for each city in france prensent in our geojson files

### 1 import the geojson file from : data.gouv

In [1]:
import geopandas as gpd

# Specify the path to your GeoJSON file
file_path = r'C:\Users\eurls\Downloads\a-com2022-geo-2154-1.geojson'

# Load the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file(file_path)


In [2]:
#display the first five rows of the geodf
gdf.head(5)

,codgeo,dep,reg,xcl2154,ycl2154,libgeo,geometry
0,01001,01,84,848241,6563021,L'Abergement-Clémenciat,"POLYGON ((846981.43602 6564107.35993, 846822.9..."
1,01002,01,84,887495,6548152,L'Abergement-de-Varey,"POLYGON ((886027.65133 6548170.89982, 886971.3..."
2,01004,01,84,882724,6542583,Ambérieu-en-Bugey,"POLYGON ((884889.40373 6539629.93086, 885076.7..."
3,01005,01,84,847277,6545791,Ambérieux-en-Dombes,"POLYGON ((845951.24990 6547999.07815, 846080.9..."
4,01006,01,84,902191,6519791,Ambléon,"POLYGON ((901336.36121 6521123.30490, 900803.2..."


In [7]:
#extract the city name from  gfd
city_name=gdf['libgeo']
#display the first 5 element in the list city_name
city_name[0:5]

0    L'Abergement-Clémenciat
1      L'Abergement-de-Varey
2          Ambérieu-en-Bugey
3        Ambérieux-en-Dombes
4                    Ambléon
Name: libgeo, dtype: object

### 2) extract the density population from wikipedia , as wikepedia in not a governemental official website we cannot  rely on it , the best is to contact the governement office responsible for the data we need

###  NOTE : the following code is particularly long to run ,it took me almost 5 hours as it have to exctract the data from more than 36 000 webpages 
### you might be able to use pyspark or some other  bigdata tools to speed up the process 

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# list
city_name

# Function to extract population density from Wikipedia
def get_population_density(city_name):
    url = f"https://fr.wikipedia.org/wiki/{city_name.replace(' ', '_')}"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the <td> element containing the population density information
    density_th = soup.find('th', string='Densité')
    
    if density_th:
        density_td = density_th.find_next('td')
        
        if density_td:
            # Extract and clean the text content within the element
            density_text = density_td.get_text(strip=True)
            
            # Concatenate all parts obtained from splitting the text
            number_parts = density_text.split()
            full_number = ' '.join(number_parts)
            
            return full_number
    
    return None



# Create an empty list to store population densities
density_list = []

# Iterate through the cities in the dataframe and append the results to the list
for city in city_name:
    density = get_population_density(city)
    density_list.append(density)

# Print the list of population densities
print(density_list)


In [ ]:
# count the number of none value 
none_count = density_list.count(None)
none_count

### observations : you can see that we got 5295 missing values ,the problem is that some french cities have the same name ,we can  improve the code to better handle this case

###  Due to the long running time ,i am out 

In [ ]:
# transform the list into a dataframe
df = pd.DataFrame(density_list)
df.head(10)

In [ ]:
#Download the list as a csv file
file_path = r"C:\Users\eurls\Downloads\density.csv"
df.to_csv(file_path, index=False)

###  if you want to use it for some calculation keep in mind that you have to clean it 
### replace the ',' by ".", remove non-numeric characters (you got some space between some values,convert it to float or int  ),